# Clinical Data Processing

## Where the data at?

In [1]:
input_path = '../Data/Intermediate_Files/'
clinicaldata_path = '../Data/Raw_Data/Clinical_Data/'
output_path = '../Data/Processed_Data/'

## Load Methyl Data

In [2]:
import pandas as pd

df_methyl = pd.read_pickle(
    input_path+'2_MethylData_Processing_Output.pkl')

# .T.reset_index(level=0, names='Batch')

print(
    f' Dataset (df) contains {df_methyl.shape[1]} columns (mC sites) and {df_methyl.shape[0]} rows (samples).')


 Dataset (df) contains 333249 columns (mC sites) and 3357 rows (samples).


In [3]:
df_methyl['Batch'].value_counts(dropna=False)

Batch
GSE49031          933
GSE190931         581
GSE124413         495
GSE159907         316
GDC_TARGET-AML    314
GDC_TCGA-AML      194
GSE152710         166
GSE147667         153
GDC_TARGET-ALL    141
GSE133986          64
Name: count, dtype: int64

## Add Labels/Clinical Outcome Data

In [4]:
# Import functions to clean up clinical data
from FM_Functions.Clinical_Data_CleanUp import *

# Combine all clinical data files into one dataframe and indexes it by the sample ID
labels_cog, labels_aml02, labels_aml08, labels_aml05 = combine_and_index_clinicaldata()
# Clean up and adjust clinical data labels
labels_aml02 = clean_aml02(labels_aml02)
labels_aml08 = clean_aml08(labels_aml08)
labels_cog   = clean_cog(labels_cog)
labels_aml05 = clean_aml05(labels_aml05)

# Combine all clinical data labels
df = pd.concat([labels_aml02, labels_aml08, labels_cog,
               labels_aml05], axis=0, join='outer')

# Remove samples that are not in the methyl dataset
# df = df.loc[df.index.isin(df_methyl.index)]

# Label control samples from the AML0531 clinical trial (GSE124413) as 'Bone Marrow Normal'

def label_control_samples(df_methyl, df):
    """
    This function labels control samples from the AML0531 clinical trial (GSE124413) as 'Bone Marrow Normal'
    and combines them with the clinical trial samples.
    """
    a = df_methyl[df_methyl['Batch'].isin(['GSE124413'])]
    b = df[df.index.isin(a.index)]
    control_0531 = a[~a.index.isin(b.index)]
    control_0531['Sample Type'] = 'Bone Marrow Normal'
    df_ = pd.concat(
        [df, control_0531['Sample Type'].to_frame()], axis=0, join='outer')
    return df_

df_ = label_control_samples(df_methyl, df)


In [5]:
df.shape

(2282, 291)

## MDS_tAML

In [114]:
meta = pd.read_pickle('../Data/Raw_Data/Methyl_Array_450k/GSE152710/sample_sheet_meta_data.pkl').iloc[:,:-1].set_index('Sample_ID')

## Nordic ALL

In [134]:
meta = pd.read_pickle('../Data/Raw_Data/Methyl_Array_450k/GSE49031/sample_sheet_meta_data.pkl').iloc[:,:-1].set_index('Sample_ID')

## Tcell_ALL_GRAAL

In [142]:
meta = pd.read_pickle('../Data/Raw_Data/Methyl_Array_EPIC/GSE147667/sample_sheet_meta_data.pkl').iloc[:,:-1].set_index('Sample_ID')

In [145]:
meta.head()

,source,age,gender,diagnosis,tissue,description,Sentrix_ID,Sentrix_Position,Sample_Group,Sample_Name,Sample_Plate,Sample_Type,Sub_Type,Sample_Well,Pool_ID,GSM_ID,Control
Sample_ID,,,,,,,,,,,,,,,,,
201233650075_R07C01,Healthy Thymus,NA,NA,normal,Normal Thymic cell type,genomic data from CD34 positive thymic cells r...,201233650075,R07C01,None,CD34_1,None,Unknown,None,None,None,GSM4439759,False
203724130020_R01C01,Healthy Thymus,NA,NA,normal,Normal Thymic cell type,genomic data from CD34 positive thymic cells r...,203724130020,R01C01,None,CD34_2,None,Unknown,None,None,None,GSM4439760,False
201233650075_R08C01,Healthy Thymus,NA,NA,normal,Normal Thymic cell type,genomic data from Immature Simple positive CD4...,201233650075,R08C01,None,ISP_1,None,Unknown,None,None,None,GSM4439761,False
203724130020_R02C01,Healthy Thymus,NA,NA,normal,Normal Thymic cell type,genomic data from Immature Simple positive CD4...,203724130020,R02C01,None,ISP_2,None,Unknown,None,None,None,GSM4439762,False
201233650023_R05C01,Healthy Thymus,NA,NA,normal,Normal Thymic cell type,genomic data from mature CD4 positive thymic c...,201233650023,R05C01,None,SPo4_1,None,Unknown,None,None,None,GSM4439764,False


In [147]:
meta['description'].value_counts()

description
genomic data from T-ALL leukemic blasts                                     143
genomic data from CD34 positive thymic cells rep-1                            1
genomic data from CD34 positive thymic cells rep-2                            1
genomic data from Immature Simple positive CD4+ thymic cells rep-1            1
genomic data from Immature Simple positive CD4+ thymic cells rep-2            1
genomic data from mature CD4 positive thymic cells rep-1                      1
genomic data from mature CD4 positive thymic cells rep-2                      1
genomic data from mature CD8 positive thymic cells rep-1                      1
genomic data from mature CD8 positive thymic cells rep-2                      1
genomic data from Double Positive CD4+ CD8+ TCR- CD3- thymic cells rep-1      1
genomic data from Double Positive CD4+ CD8+ TCR- CD3- thymic cells rep-2      1
genomic data from Double Positive CD4+ CD8+ TCR+ CD3+ thymic cells rep-1      1
genomic data from Double Pos

## GDC TARGET ALL

In [129]:
# load clinical data from GDC
clinical_tsv = pd.read_csv('../Data/Raw_Data/Methyl_Array_EPIC/GDC_TARGET-ALL/clinical.tsv', 
                            sep='\t', index_col=0)[['case_submitter_id']].drop_duplicates()

In [131]:
clinical_tsv

,case_submitter_id
case_id,
92aa6eba-2d43-4b40-9467-305bde60666b,TARGET-15-SJMPAL003414
fde8ad74-7071-42b7-b47a-8227fe049f1f,TARGET-15-SJMPAL046466
6a39937a-0e74-4637-b77c-c3fa4161b47c,TARGET-15-SJMPAL046471
5aab9dc1-efdb-4eb8-ab7b-d0569e233698,TARGET-20-SJAML045737
a018a6bf-9e9c-4076-9e2e-7916c17483ce,TARGET-15-SJMPAL012427
af5b5876-e966-4ca3-96ab-84c8dd6adb7c,TARGET-15-SJMPAL040024
6deec82d-51cc-4c60-a445-20265889fe78,TARGET-15-SJMPAL016447
1bdd092b-b5dc-4626-b39d-c8c4532af8cc,TARGET-20-SJAML003322
e8116a9a-0b3e-4c5d-a895-432806794124,TARGET-20-SJAML045742


## TCGA AML

In [105]:
def merge_index_amltcga():

    # load clinical data from GDC
    clinical_tsv = pd.read_csv('../Data/Raw_Data/Methyl_Array_450k/GDC_TCGA-AML/clinical.tsv', 
                    sep='\t', index_col=0)[['case_submitter_id']].drop_duplicates()

    # extract last 4 digits from case_id to get TCGA Patient ID
    clinical_tsv['TCGA Patient ID'] = clinical_tsv['case_submitter_id'].str[-4:]

    # set index to TCGA Patient ID
    clinical_tsv = clinical_tsv.reset_index().set_index('TCGA Patient ID').sort_index()

    # load meta data from NEJM 2013 paper
    meta = pd.read_excel('../Data/Raw_Data/Clinical_Data/TCGA_LAML/SuppTable01_NEJM2013_TCGA_AML.Paper_Mutation data.xlsx',
                        index_col=1).iloc[1:,:].sort_index()

    # make meta index integers
    meta.index = meta.index.astype(int)
    clinical_tsv.index = clinical_tsv.index.astype(int)

    # join clinical_tsv and meta
    labels_amltcga = clinical_tsv.join(meta, how='left')

    # set index to case_id
    labels_amltcga = labels_amltcga.reset_index().set_index('case_id')
    
    return labels_amltcga


## BeatAML Clinical Data

In [39]:
import pandas as pd
def merge_index_beataml ():
    meta = pd.read_pickle('../Data/Raw_Data/Methyl_Array_EPIC/GSE159907/sample_sheet_meta_data.pkl').iloc[:,:-1]

    # Create a new column with only the content inside [] from column 'Sample_Name'
    meta['LLS_SampleID'] = meta['Sample_Name'].str.extract(r"\[(.*?)\]", expand=False)

    # Set the index to the new column
    meta1 = meta[['tissue','disease_state','LLS_SampleID','Sample_ID']].set_index('LLS_SampleID')

    # Read in the clinical data
    meta2 = pd.read_excel('../Data/Raw_Data/Clinical_Data/BeatAML/BEAT_AML_Raw clinical data_702.Samples.Vizome.xlsx', index_col=3)

    # Join the two dataframes
    labels_beataml = meta1.join(meta2, how='left').reset_index().set_index('Sample_ID')

    return labels_beataml

labels_beataml = clean_beataml()

## Remove Samples based on Certain Clinical Features

### Remove Relapse Samples

In [5]:
df1 = df_[~df_['Sample Type'].isin(['Relapse', 'Recurrent Blood Derived Cancer - Bone Marrow',
                                    'Recurrent Blood Derived Cancer - Peripheral Blood'])]

print(
    f'Out of {df_.shape[0]} samples, {df_.shape[0]-df1.shape[0]} matched, yielding {df1.shape[0]} samples after filtering')


Out of 1762 samples, 248 matched, yielding 1514 samples after filtering


### Remove Control/Normal Samples

In [6]:
df2 = df1[~df1['Sample Type'].isin(
    ['Bone Marrow Normal', 'Blood Derived Normal'])]
print(
    f'Out of {df1.shape[0]} samples, {df1.shape[0]-df2.shape[0]} matched, yielding {df2.shape[0]} samples after filtering')


Out of 1514 samples, 154 matched, yielding 1360 samples after filtering


### Remove Duplicate Samples

In [7]:
df3 = df2[~df2['Patient_ID'].duplicated(keep='last')]
print(
    f'Out of {df2.shape[0]} samples, {df2.shape[0]-df3.shape[0]} matched, yielding {df3.shape[0]} samples after filtering')


Out of 1360 samples, 14 matched, yielding 1346 samples after filtering


## Save Files

In [8]:
output = df3.join(df_methyl,how='left') # Join clinical data with methyl data

x = output.iloc[:,df3.shape[1]+1:] # Select only methyl data
y = output.iloc[:,0:df3.shape[1]+1] # Select only clinical data

In [9]:
# Split train and test by clinical trial
y_train = y[~y['Clinical Trial'].isin(['AML02', 'AML08'])]
# y_train = y_train[y_train['Sample Type'].isin(['Diagnosis',
#        'Primary Blood Derived Cancer - Bone Marrow', 'Bone Marrow Normal',
#        'Primary Blood Derived Cancer - Peripheral Blood',
#        'Blood Derived Normal'])]

y_test = y[y['Clinical Trial'].isin(['AML02', 'AML08'])]

# Select samples in x that are in y_train
x_train = x.loc[y_train.index]
x_test = x.loc[y_test.index]

# x_train = pd.concat([x_train, ctrl_x], axis=0)
# y_train = pd.concat([y_train, ctrl_y], axis=0,keys=['Diagnosis','Control'], names=['sample_type'])


print(
    f"Discovery dataset (train) contains {x_train.shape[1]} rows (5mC sites) and {x_train.shape[0]} columns (samples)")
print(
    f"\n{y_train['Clinical Trial'].value_counts(dropna=False).to_string()}\n")
print(
    f"Validation dataset (test) contains {x_test.shape[1]} rows (5mC sites) and {x_test.shape[0]} columns (samples).")
print(f"\n{y_test['Clinical Trial'].value_counts(dropna=False).to_string()}\n")


Discovery dataset (train) contains 310545 rows (5mC sites) and 1142 columns (samples)

AAML1031    520
AAML0531    508
AML05        64
AAML03P1     36
CCG2961      14

Validation dataset (test) contains 310545 rows (5mC sites) and 204 columns (samples).

AML02    162
AML08     42



## Batch Correction with pyCombat

- __pyCombat__: a Python tool for batch effects correction in high-throughput molecular data using empirical Bayes methods

- __Github__: [https://epigenelabs.github.io/pyComBat/](https://epigenelabs.github.io/pyComBat/)

- __Implementation Paper__: [bioRxiv](https://doi.org/10.1101/2020.03.17.995431)

- __Original Paper__: [Biostatistics](https://pubmed.ncbi.nlm.nih.gov/16632515/)

In [10]:
from combat.pycombat import pycombat

# Correct batch effects in the training dataset
x_train2 = pycombat(x_train.T, y_train['Batch']).T

print('Succesfully corrected batch effects in the training dataset.')


Found 4 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
Succesfully corrected batch effects in the training dataset.


In [11]:
y_train = y_train[~y_train['Clinical Trial'].isin(['AML05'])]
x_train3 = x_train2.loc[y_train.index]


print(
    f"Discovery dataset (train) contains {x_train3.shape[1]} rows (5mC sites) and {x_train3.shape[0]} columns (samples)")
print(
    f"\n{y_train['Clinical Trial'].value_counts(dropna=False).to_string()}\n")
print(
    f"Validation dataset (test) contains {x_test.shape[1]} rows (5mC sites) and {x_test.shape[0]} columns (samples).")
print(f"\n{y_test['Clinical Trial'].value_counts(dropna=False).to_string()}\n")

Discovery dataset (train) contains 310545 rows (5mC sites) and 1078 columns (samples)

AAML1031    520
AAML0531    508
AAML03P1     36
CCG2961      14

Validation dataset (test) contains 310545 rows (5mC sites) and 204 columns (samples).

AML02    162
AML08     42



In [12]:
x_train3.to_pickle(output_path+'x_train.pkl') # Save methyl data
y_train.to_csv(output_path+'y_train.csv') # Save clinical data

print(
    f'Successfuly saved methyl data in x.pkl and clinical data in y.csv.\nPath: {output_path}')

Successfuly saved methyl data in x.pkl and clinical data in y.csv.
Path: ../Data/Processed_Data/


### Save Control and Relapse Data Separately

In [13]:
controls = df_[df_['Sample Type'].isin(['Bone Marrow Normal'])]

relapse = df_[df_['Sample Type'].isin(['Relapse', 'Recurrent Blood Derived Cancer - Bone Marrow',
                                       'Recurrent Blood Derived Cancer - Peripheral Blood'])]

# Merge control and relapse samples
t = pd.concat([controls, relapse], axis=0, join='outer',
              names=['Control', 'Relapse'])

# Join clinical data with methyl data
t2 = df_methyl.join(t, how='right')

# Save merged control and relapse samples
t2.to_pickle(output_path+'control_relapse.pkl')

print(
    f'Successfuly saved {controls.shape[0]} control samples and {relapse.shape[0]} relapse samples.\nPath: {output_path}')


Successfuly saved 147 control samples and 248 relapse samples.
Path: ../Data/Processed_Data/


## Watermark

In [14]:
%load_ext watermark

In [15]:
# produce a list of the loaded modules
%watermark -v -p pandas

Python implementation: CPython
Python version       : 3.10.10
IPython version      : 8.3.0

pandas: 1.5.3

